# Update HSIA Rasdaman data

This notebook updates the data for [Rasdaman on Zeus](https://zeus.snap.uaf.edu/rasdaman/ows). This pipeline builds on the existing data by simply updating it with the newest year of data if available - it does not produce the entire dataset.  

### 1. Download new data

Download new data from the NSIDC. These data of interest are in the [0051 dataset](https://nsidc.org/data/nsidc-0051). Change the user options as needed at the top of the following code cell.

#### NASA earthdata credentials

The user needs to store Earthdata credentials in `$HOME/.netrc`. The `.netrc` file should have the following format:

```
machine urs.earthdata.nasa.gov login myusername password mypassword
```

### 2. Run the download code below

Update the variable year to the year desired to download.

In [1]:
import os
import netrc
from urllib.parse import urlparse

# Change the year to match the year we want to download
year = 2021
out_dir = os.getenv("RAW_DIR") or f"/tmp/hsia_updates_2021/nsidc_raw/{year}"
info = netrc.netrc()
username, account, password = info.authenticators(urlparse("https://urs.earthdata.nasa.gov").hostname)

if not os.path.exists( out_dir ):
        os.makedirs( out_dir )

for month in ["{:02d}".format(month + 1) for month in range(12)]:
    commanda = 'wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --keep-session-cookies --no-check-certificate --auth-no-challenge=on -r --reject "index.html*" -np -nd -e robots=off --user {} --password "{}" -P {} '.format(
        username, password, out_dir
    )
    commandb = "https://daacdata.apps.nsidc.org/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/monthly/nt_{}{}_f17_v1.1_n.bin".format(
        str(year), month
    )
    os.system(commanda + commandb)

--2022-08-03 15:14:27--  https://daacdata.apps.nsidc.org/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/monthly/nt_202101_f17_v1.1_n.bin
Resolving daacdata.apps.nsidc.org (daacdata.apps.nsidc.org)... 128.138.135.43
Connecting to daacdata.apps.nsidc.org (daacdata.apps.nsidc.org)|128.138.135.43|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 136492 (133K) [application/octet-stream]
Saving to: ‘/tmp/hsia_updates_2021/nsidc_raw/2021/nt_202101_f17_v1.1_n.bin’

     0K .......... .......... .......... .......... .......... 37%  372K 0s
    50K .......... .......... .......... .......... .......... 75%  750K 0s
   100K .......... .......... .......... ...                  100% 21.7M=0.2s

2022-08-03 15:14:27 (658 KB/s) - ‘/tmp/hsia_updates_2021/nsidc_raw/2021/nt_202101_f17_v1.1_n.bin’ saved [136492/136492]

FINISHED --2022-08-03 15:14:27--
Total wall clock time: 0.6s
Downloaded: 1 files, 133K in 0.2s (658 KB/s)
--2022-0

     0K .......... .......... .......... .......... .......... 37%  373K 0s
    50K .......... .......... .......... .......... .......... 75%  754K 0s
   100K .......... .......... .......... ...                  100% 14.8M=0.2s

2022-08-03 15:14:31 (658 KB/s) - ‘/tmp/hsia_updates_2021/nsidc_raw/2021/nt_202108_f17_v1.1_n.bin’ saved [136492/136492]

FINISHED --2022-08-03 15:14:31--
Total wall clock time: 0.5s
Downloaded: 1 files, 133K in 0.2s (658 KB/s)
--2022-08-03 15:14:31--  https://daacdata.apps.nsidc.org/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/monthly/nt_202109_f17_v1.1_n.bin
Resolving daacdata.apps.nsidc.org (daacdata.apps.nsidc.org)... 128.138.135.43
Connecting to daacdata.apps.nsidc.org (daacdata.apps.nsidc.org)|128.138.135.43|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 136492 (133K) [application/octet-stream]
Saving to: ‘/tmp/hsia_updates_2021/nsidc_raw/2021/nt_202109_f17_v1.1_n.bin’

     0K 

### 3. Generate HSIA TIF files in EPSG:3572 

In [2]:
from hsia.seaice import SeaIceRaw
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import glob

def main(fn, output_path):
    sic = SeaIceRaw(fn)
    
    # Collect the year and month from the file name from NSIDC data.
    year_month = str(os.path.basename(fn)).split('_')[1]
    
    # Output filename is generated with year + month.
    output_filename = os.path.join( output_path, f'seaice_conc_sic_mean_pct_monthly_panarctic_{year_month[0:4]}_{year_month[4:]}.tif')
    
    # Run GDAL to warp file to match previous years in hsia_arctic_production
    # Rasdaman coverage.
    sic.to_gtiff_3572(output_filename)
    
years = [2021]
for year in years:
    base_path = f'/tmp/hsia_updates_{year}'
    dat_path = os.path.join( base_path, 'nsidc_raw', str(year) )
    output_path = os.path.join( base_path, f"hsia_prepped_{year}" )
    if not os.path.exists( output_path ):
        os.makedirs( output_path )
    l = glob.glob( os.path.join( dat_path, '*.bin' ) )
    _ = [ main(fn, output_path ) for fn in l ]

### 4. Combine new files with previous years

On poseidon.snap.uaf.edu, we want to update the previous years' data with the new year's data.

***poseidon*** $ rsync -av /workspace/Shared/Tech_Projects/Sea_Ice_Atlas/final_products/rasdaman_hsia_arctic_production_tifs.tgz apollo.snap.uaf.edu:/tmp/hsia_updates_2021/

***apollo*** $ cd /tmp/hsia_updates_2021/

***apollo*** $ tar zxf rasdaman_hsia_arctic_production_tifs.tgz

***apollo*** $ mv /tmp/hsia_updates_2021/hsia_prepped_2021/* /tmp/hsia_updates_2021/rasdaman_hsia_arctic_production_tifs/

***apollo*** $ tar zcf rasdaman_hsia_arctic_production_tif.tgz rasdaman_hsia_arctic_production_tifs

***apollo*** $ rsync -av rasdaman_hsia_arctic_production_tifs.tgz poseidon.snap.uaf.edu:/workspace/Shared/Tech_Projects/Sea_Ice_Atlas/final_productions/rasdaman_hsia_arctic_production_tifs.tgz

### 5. Ingest TIFs into Rasdaman coverage on Apollo

On Apollo, we will now ingest all of the TIFs from 1850-present day for sea ice concentration for the coverage ***hsia_arctic_production***.

1. First, remove the previous coverage: 

    wget "http://localhost:8080/rasdaman/ows?SERVICE=WCS&VERSION=2.0.1&REQUEST=DeleteCoverage&COVERAGEID=hsia_arctic_production"
    
    
2. Clone the Rasdaman ingest Git repository:

    git clone https://github.com/ua-snap/rasdaman-ingest.git
    
    
3. Source the Rasdaman profile:

    source /etc/profile.d/rasdaman.sh
  
  
4. Run the hsia_ingest_arctic.json file:

    wcst_import.sh rasdaman-ingest/hsiaa/hsia_ingest_arctic.json